In [1]:
# <center> **Практика**

## <center> **Прогнозирование биологического ответа**

Необходимо предсказать биологический ответ молекул (столбец 'Activity') по их химическому составу (столбцы D1-D1776).

Первый столбец Activity содержит экспериментальные данные, описывающие фактический биологический ответ [0, 1]; 
Остальные столбцы D1-D1776 представляют собой молекулярные дескрипторы — это вычисляемые свойства, которые могут фиксировать некоторые характеристики молекулы, например размер, форму или состав элементов.

Предварительная обработка не требуется, данные уже закодированы и нормализованы.

В качестве метрики будем использовать F1-score.

Необходимо обучить две модели: логистическую регрессию и случайный лес. Далее нужно сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации. Важно использовать все четыре метода (GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna) хотя бы по разу, максимальное количество итераций не должно превышать 50.

In [9]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных

from sklearn import linear_model #линейные модели
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки
from sklearn.model_selection import cross_val_score

Загрузка данных

In [3]:
data = pd.read_csv('_train_sem09 (1).csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# cоздание матрицы наблюдений X и вектора ответов y
X = data.drop(['Activity'], axis=1)
y = data['Activity']

# Разделение выборки на тренировочную и тестовую в соотношении 80/20
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2)

---

## <center> **Логистическая регрессия**

## Базовая модель с параметрами по умолчанию

In [22]:
log_reg = linear_model.LogisticRegression(max_iter = 1000) # cоздаем объект класса логистическая регрессия
log_reg.fit(X_train, y_train) # обучаем модель
y_test_pred = log_reg.predict(X_test) # делаем предсказание
print(f'F1-score базовой логистической регрессии: {metrics.f1_score(y_test, y_test_pred).round(3)}') # вывод метрики

F1-score базовой логистической регрессии: 0.789


## Подберем гиперпараметры с помощью GridSeachCV

In [9]:
from sklearn.model_selection import GridSearchCV

# укажем гиперпараметры и их значение в словаре
param_grid = {'penalty': ['l2', 'none'] , # тип регурялизации
              'solver': ['lbfgs', 'saga'] # алгоритм оптимизации
              }
# инициируем класс, передадим в него словарь, модель
grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state=42, # генератор случайных чисел
        max_iter=1000 # количество итераций на сходимость
    ), 
    param_grid=param_grid, # гиперпараметры
    cv=5, # фолды для кросс-валидации
    n_jobs = -1 # используем все дотупные ядра
)
  
%time grid_search.fit(X_train, y_train) # обучение модели с указанием затраченного времени
y_test_pred = grid_search.predict(X_test) # предсказание на тестовой выборке

print(f'F1-score логистической регрессии c GridSearchCV: {metrics.f1_score(y_test, y_test_pred).round(3)}') 
print(f'Наилучшие значения гиперпараметров GridSearchCV: {grid_search.best_params_}')

CPU times: total: 15.5 s
Wall time: 6min 52s
F1-score логистической регрессии c GridSearchCV: 0.789
Наилучшие значения гиперпараметров GridSearchCV: {'penalty': 'l2', 'solver': 'lbfgs'}


Метрика не изменилась, да и времени на работу алгоритма потребовалось 
не так уж и много - 6.5 мин.
Попробуем расширить сетку гиперпараметров, добавив силу регуляризации.

In [10]:
# укажем гиперпараметры и их значение в словаре
param_grid = {'penalty': ['l2', 'none'] , # тип регурялизации
              'solver': ['lbfgs', 'saga'], # алгоритм оптимизации
              'C': [0.01, 0.03, 0.05, 0.08] # сила регуляризации
              }
# инициируем класс, передадим в него словарь, модель
grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state=42, # генератор случайных чисел
        max_iter=1000 # количество итераций на сходимость
    ), 
    param_grid=param_grid, # гиперпараметры
    cv=5, # фолды для кросс-валидации
    n_jobs = -1 # используем все дотупные ядра
)
  
%time grid_search.fit(X_train, y_train) # обучение модели с указанием затраченного времени
y_test_pred = grid_search.predict(X_test) # предсказание на тестовой выборке

print(f'F1-score логистической регрессии c GridSearchCV: {metrics.f1_score(y_test, y_test_pred).round(3)}') 
print(f'Наилучшие значения гиперпараметров GridSearchCV: {grid_search.best_params_}')

CPU times: total: 4.98 s
Wall time: 17min 28s
F1-score логистической регрессии c GridSearchCV: 0.793
Наилучшие значения гиперпараметров GridSearchCV: {'C': 0.05, 'penalty': 'l2', 'solver': 'lbfgs'}


Метрику удалось повысить, но потребовалось в 3 раза больше времени времени.

## Подберем гиперпараметры с помощью RandomizedSearchCV

In [11]:
from sklearn.model_selection import RandomizedSearchCV

# # укажем гиперпараметры и их значение в словаре
param_distributions = {'penalty': ['l2', 'none'], # тип регурялизации
                       'solver': ['lbfgs', 'saga'], # алгоритм оптимизации
                       'C': list(np.linspace(0.03, 0.1, 20, dtype=float)) # диапазон силы регуляризации
                    }

# инициируем класс, передадим в него словарь, модель            
random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state=42, # воспроизводимость
        max_iter=1000 # количество итераций на сходимость
        ), 
    param_distributions=param_distributions, # гиперпараметры
    cv=5, # фолды для кросс-валидации
    n_iter = 10, # количество комбинаций на расчет
    n_jobs = -1 # используем все дотупные ядра
)  
%time random_search.fit(X_train, y_train) # обучение модели с указанием затраченного времени
y_test_pred = random_search.predict(X_test) # предсказание на тестовой выборке

print(f'F1-score логистической регрессии c RandomizedSearchCV: {metrics.f1_score(y_test, y_test_pred).round(3)}') 
print(f'Наилучшие значения гиперпараметров RandomizedSearchCV: {random_search.best_params_}')

CPU times: total: 5.25 s
Wall time: 10min 43s
F1-score логистической регрессии c RandomizedSearchCV: 0.796
Наилучшие значения гиперпараметров RandomizedSearchCV: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.04105263157894737}


Метрику удалось ещё чуть-чуть улучшить, при этом времени понадобилось меньшe.

## Подберем гиперпараметры с помощью Hyperopt

In [11]:
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials

# зададим пространство поиска гиперпараметров
pen = ['l2', 'none']
sol = ['lbfgs', 'saga']
space={'penalty': hp.choice('penalty', ['l2', 'none']), # тип регурялизации
       'solver' : hp.choice('solver', ['lbfgs', 'saga']), # алгоритм оптимизации
       'C': hp.uniform('C', low=0.03, high=0.08) # диапазон силы регуляризации
}
random_state = 42 # зафксируем random_state
# создадим функцию, которая будет считать метрику
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'penalty': params['penalty'], 
              'solver': params['solver'], 
             'C': float(params['C'])
              }
  
    # используем эту комбинацию для построения модели
    model = linear_model.LogisticRegression(**params, random_state=random_state)

    model.fit(X, y) # обучаем модель
    # считаем метрику на модели, обученной с помощью кросс-валидации
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()
    # алгоритм работает только на минимизацию, поэтому ставим знак минус
    return-score

%time # выведем время работы алгоритма
trials = Trials() # логируем результаты

# подбор гиперпараметров
best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state) # воспроиводимость
         )

# создадим модель с подобранными гиперпараметрами
model = linear_model.LogisticRegression(
    random_state=random_state,
    penalty=pen[best['penalty']],
    solver=sol[best['solver']],
    C=float(best['C'])
    )
model.fit(X_train, y_train) # обучение модели
y_test_pred = model.predict(X_test) # предсказание на тестовой выборке

print(f'F1-score логистической регрессии c Hyperopt: {metrics.f1_score(y_test, y_test_pred).round(3)}') 
print(f'Наилучшие значения гиперпараметров Hyperopt: {best}')

TPE is being used as the default algorithm.


CPU times: total: 0 ns
Wall time: 0 ns
  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

c:\Users\mors0\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(

c:\Users\mors0\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



  5%|▌         | 1/20 [00:13<04:07, 13.03s/trial, best loss: -0.7556086428138172]

c:\Users\mors0\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



...

c:\Users\mors0\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



 95%|█████████▌| 19/20 [07:30<00:30, 30.53s/trial, best loss: -0.7862068484370935]

c:\Users\mors0\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



100%|██████████| 20/20 [07:37<00:00, 22.88s/trial, best loss: -0.7862068484370935]
F1-score логистической регрессии c Hyperopt: 0.792
Наилучшие значения гиперпараметров Hyperopt: {'C': 0.0665019800181027, 'penalty': 0, 'solver': 0}


c:\Users\mors0\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Значение метрики осталось тем же. 

## Подберем гиперпараметры с помощью Optuna

In [10]:
import optuna

random_state = 42
# подбор гиперпараметров
def optuna_rf(trial):
    # задаем пространства поиска гиперпараметров
    penalty = trial.suggest_categorical('penalty', ['l2', 'none']) # тип регурялизации
    solver = trial.suggest_categorical('solver', ['lbfgs', 'saga']) # алгоритм оптимизации
    C = trial.suggest_float('C', 0.03, 0.08) # диапазон силы регуляризации
  
    # создаем модель
    model = linear_model.LogisticRegression(
        penalty=penalty,
        solver=solver,
        C=C,
        random_state=random_state)
    # обучаем модель
    model.fit(X_train, y_train)
    score = cross_val_score(model, X, y, cv=5, scoring="f1", n_jobs=-1).mean()

    return score

%time # отобразим время работы алгоритма
# cоздаем объект исследования
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

# модель с выбранными гиперпараметрами
model = linear_model.LogisticRegression(**study.best_params,random_state=random_state)
model.fit(X_train, y_train) # обучение модели
y_test_pred = model.predict(X_test) # предсказание на тестовой выборке

print(f'F1-score логистической регрессии c Optuna: {metrics.f1_score(y_test, y_test_pred).round(3)}') 
print(f'Наилучшие значения гиперпараметров Optuna: {study.best_params}')

[I 2023-06-15 08:18:33,257] A new study created in memory with name: LogisticRegression


CPU times: total: 0 ns
Wall time: 0 ns


c:\Users\mors0\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2023-06-15 08:19:37,931] Trial 0 finished with value: 0.7864761995512766 and parameters: {'penalty': 'l2', 'solver': 'saga', 'C': 0.06715954052147481}. Best is trial 0 with value: 0.7864761995512766.
c:\Users\mors0\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[I 2023-06-15 08:19:47,395] Trial 1 finished wi

F1-score логистической регрессии c Optuna: 0.799
Наилучшие значения гиперпараметров Optuna: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.03057159953421766}


c:\Users\mors0\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Метрика повысилась за относительно небольшое время.

## <center> **Случайный лес**

## Базовая модель с параметрами по умолчанию

In [13]:
rf = ensemble.RandomForestClassifier(random_state=42) # инициализация класса
rf.fit(X_train, y_train) # обучение модели
y_test_pred = rf.predict(X_test) # делаем предсказание
print(f'F1-score базовой модели случайного леса: {metrics.f1_score(y_test, y_test_pred).round(3)}') # вывод метрики

F1-score базовой модели случайного леса: 0.828


## Подберем гиперпараметры с помощью GridSeachCV

In [16]:
# укажем гиперпараметры и их значения в словаре
param_grid = {'n_estimators': list(range(80, 200, 30)), # количество деревьев влесу
              'min_samples_leaf': [5], # минимальное количество объектов в листе
              'max_depth': list(np.linspace(20, 40, 10, dtype=int)) # глубина деревьев
              }

# подбор гиперпараметров для модели
grid_search_forest = GridSearchCV( # инициализация рандомайзера
    estimator=ensemble.RandomForestClassifier(random_state=42), # модель
    param_grid=param_grid, # словарь параметров 
    cv=5, # количество фолдов для кросс-валидации
    n_jobs = -1 # задействуем все ядра
) 
# обучение модели с выбранными гиперпараметрами, указав время
%time grid_search_forest.fit(X_train, y_train)
# сделаем предсказание на тестовой выборке
y_test_pred = grid_search_forest.predict(X_test)

print(f'F1-score случайного леса c GridSearchCV: {metrics.f1_score(y_test, y_test_pred).round(3)}') 
print(f'Наилучшие значения гиперпараметров GridSearchCV: {grid_search_forest.best_params_}')

CPU times: total: 3.75 s
Wall time: 3min 21s
F1-score случайного леса c GridSearchCV: 0.843
Наилучшие значения гиперпараметров GridSearchCV: {'max_depth': 22, 'min_samples_leaf': 5, 'n_estimators': 140}


Метрику удалось повысить, времени понадобилось немного.

## Подберем гиперпараметры с помощью RandomizedSearchCV

In [18]:
from sklearn.model_selection import RandomizedSearchCV

# укажем гиперпараметры и их значения в словаре
param_distributions = {'n_estimators': list(range(80, 200, 30)), # количество деревьев влесу
              'min_samples_leaf': [5], # минимальное количество объектов в листе
              'max_depth': list(np.linspace(20, 40, 10, dtype=int)) # глубина деревьев
              }

# подбор гиперпараметров для модели
random_search_forest = RandomizedSearchCV( # инициализация рандомайзера
    estimator=ensemble.RandomForestClassifier(random_state=42), # модель
    param_distributions=param_distributions, # словарь параметров 
    cv=5, # количество фолдов для кросс-валидации
    n_iter = 10, # количество комбинаций на расчет
    n_jobs = -1 # задействуем все ядра
) 
# обучаем модель с подобранными гиперпараметрами
%time random_search_forest.fit(X_train, y_train)
# предсказание на тестовой выборке
y_test_pred = random_search_forest.predict(X_test)

print(f'F1-score случайного леса c RandomizedSearchCV: {metrics.f1_score(y_test, y_test_pred).round(3)}') 
print(f'Наилучшие значения гиперпараметров GridSearchCV: {grid_search_forest.best_params_}')

CPU times: total: 2.89 s
Wall time: 57.5 s
F1-score случайного леса c RandomizedSearchCV: 0.833
Наилучшие значения гиперпараметров GridSearchCV: {'max_depth': 22, 'min_samples_leaf': 5, 'n_estimators': 140}


Времени понадобилось совсем немного, но метрика упала.

## Подберем гиперпараметры с помощью Hyperopt

In [19]:
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials

# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 200, 1), # количество деревьев в лесу
       'max_depth' : hp.quniform('max_depth', 15, 26, 1), # максимальная глубина дерева
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1) # мин. кол-во объктов в влисте
      }
random_state = 42

# подсчет значения метрики при различных комбинациях гиперпараметров
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # постороим модель с подобранными гиперпараметрами
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    # оценим модель с помощью кросс-валидации
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()
    
    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

%time # выведем время работы алгоритма
trials = Trials() # логируем результаты

# подбор лучших значений гиперпараметров
best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )

# модель с выбранными гиперпараметрами
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_train, y_train) # обучение модели
y_test_pred = model.predict(X_test) # предсказание на тестовой выборке

print(f'F1-score случайного леса c Hyperopt: {metrics.f1_score(y_test, y_test_pred).round(3)}') 
print(f'Наилучшие значения гиперпараметров Hyperopt: {grid_search_forest.best_params_}')

CPU times: total: 0 ns
Wall time: 0 ns
100%|██████████| 20/20 [03:03<00:00,  9.19s/trial, best loss: -0.8075339330991621]
F1-score случайного леса c Hyperopt: 0.835
Наилучшие значения гиперпараметров Hyperopt: {'max_depth': 22, 'min_samples_leaf': 5, 'n_estimators': 140}


Метрику повысить  не удалось, но работал алгоритм недолго.

## Подберем гиперпараметры с помощью Optuna

In [21]:
# создание функции для расчета значения метрики при различных комбинациях гиперпараметров
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1) # количество деревьев в лесу
  max_depth = trial.suggest_int('max_depth', 15, 26, 1) # макс. глубина деревьев
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1) # мин. кол-во объектов в листе

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  # оценим модель с помощью кросс-валидации
  score = cross_val_score(model, X, y, cv=5, scoring="f1", n_jobs=-1).mean()

  return score

%time # фиксируем время
# cоздаем объект исследования, указав, что метрику надо максимизировать
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

# создадим модель с подобранными гиперпараметрами
model = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state)
model.fit(X_train, y_train) # обучение модели
y_test_pred = model.predict(X_test) # предсказание на тестовой выборке

print(f'F1-score случайного леса c Optuna: {metrics.f1_score(y_test, y_test_pred).round(3)}') 
print(f'Наилучшие значения гиперпараметров Optuna: {grid_search_forest.best_params_}')

[I 2023-06-15 09:59:45,811] A new study created in memory with name: RandomForestClassifier


CPU times: total: 0 ns
Wall time: 0 ns


[I 2023-06-15 10:00:06,504] Trial 0 finished with value: 0.8134317768787083 and parameters: {'n_estimators': 194, 'max_depth': 22, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.8134317768787083.
[I 2023-06-15 10:00:22,261] Trial 1 finished with value: 0.8086460264334576 and parameters: {'n_estimators': 197, 'max_depth': 19, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.8134317768787083.
[I 2023-06-15 10:00:29,827] Trial 2 finished with value: 0.794773451980911 and parameters: {'n_estimators': 109, 'max_depth': 20, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.8134317768787083.
[I 2023-06-15 10:00:39,091] Trial 3 finished with value: 0.7950437918182063 and parameters: {'n_estimators': 138, 'max_depth': 25, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.8134317768787083.
[I 2023-06-15 10:00:49,262] Trial 4 finished with value: 0.7964196017927637 and parameters: {'n_estimators': 149, 'max_depth': 26, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.81

F1-score случайного леса c Optuna: 0.83
Наилучшие значения гиперпараметров Optuna: {'max_depth': 22, 'min_samples_leaf': 5, 'n_estimators': 140}


Значение метрики осталось тем же, а времени понадобилось больше. 

In [ ]:
# ДЛЯ НАГЛЯДНОСТИ СОБЕРЕМ РЕЗУЛЬТАТЫ В DataFfame

In [28]:
df = pd.DataFrame({
   'F1-score': [0.789, 0.793, 0.796, 0.792, 0.799, 0.828, 0.843, 0.833, 0.835, 0.830],
'time': ['6s', '17min 28s', '10min 43s', '11min 06s', '6min 11s', '2.5s', '3min 22s', '57s', '3min 07s', '4min 24s'],
'penalty(LR) or n_estimators(RF)': ['default', 'l2', 'l2', 'l2', 'l2', 'default', 140, 140, 140, 140],
'solver(LR) or max_depth(RF)': ['default', 'lbfgs', 'lbfgs', 'lbfgs', 'lbfgs', 'default', 22, 22, 22, 22],
'C(LR) or min_samples_leaf(RF)': ['default', 0.05, 0.04, 0.06, 0.03,'default', 5, 5, 5, 5]
})
df.index = ['LogisticRegression', 'LR_GridSeachCV', 'LR_RandomizedSearchCV', 'LR_Hyperopt', 'LR_Optuna',
'RandomForest', 'RF_GridSeachCV', 'RF_RandomizedSearchCV', 'RF_Hyperopt', 'RF_Optuna']


display(df)

,F1-score,time,penalty(LR) or n_estimators(RF),solver(LR) or max_depth(RF),C(LR) or min_samples_leaf(RF)
LogisticRegression,0.789,6s,default,default,default
LR_GridSeachCV,0.793,17min 28s,l2,lbfgs,0.05
LR_RandomizedSearchCV,0.796,10min 43s,l2,lbfgs,0.04
LR_Hyperopt,0.792,11min 06s,l2,lbfgs,0.06
LR_Optuna,0.799,6min 11s,l2,lbfgs,0.03
RandomForest,0.828,2.5s,default,default,default
RF_GridSeachCV,0.843,3min 22s,140,22,5
RF_RandomizedSearchCV,0.833,57s,140,22,5
RF_Hyperopt,0.835,3min 07s,140,22,5
RF_Optuna,0.830,4min 24s,140,22,5


Мы видим, что в целом случайный лес справился с задачей лучше, чем логическая регрессия. 
Хотя все 4 метода подобрали одинаковые гиперпараметры, наилучшие результаты показала модель RandomForest в паре с методом RF_GridSeachCV.